<div style=background-color:#EEEEFF>

## 3. Recognizing "Real" vs. "Fake" jokes

In [2.FakePunchlines](2.FakePunchlines.ipynb), we got GPT-2 to generate the punchlines to some jokes.  A human can pretty easily tell which are the real punchlines and which are the GPT-2-generated ones.  But can GPT-2 fool another AI?
    
In this Notebook, we'll use the HugginFace [transformers](https://github.com/huggingface/transformers) library to train an NLP classifier to distinguish between the real joke punchlines and the fake ones generated by GPT-2.  

<div style=background-color:#EEEEFF>

We start by loading in our training and test datasets that were generated in [1.JokesDataset](1.JokesDataset.ipynb).  While we're trying to get things to work, let's downsample the data by 100x (i.e., use only 1% of the data), just so that things will run fast.  When we're ready to train for real, we'll use a factor of 1x (no downsampling).

In [2]:
# Load our dataset of real and fake jokes, split into training and test sets

from datasets import load_dataset

# For development purposes, we downsample by 100x, just so things run fast
downsample_factor = 100
# Load the data, filter out any non-string input (it breaks the code), and downsample
dataset = load_dataset('csv', data_files={'train':['data/short_jokes_train.csv','data/short_jokes_train_fake.csv'],
                                          'test':['data/short_jokes_test.csv','data/short_jokes_test_fake.csv']})
dataset = dataset.filter(lambda ex,j: ((type(ex['setup'])==str) & (type(ex['punchline'])==str) & 
                                       (j%downsample_factor==0)),                         
                         with_indices=True)
print('{} rows in the train dataset ({}x downsampled).'.format(dataset['train'].num_rows,downsample_factor))
print('{} rows in the test dataset ({}x downsampled).'.format(dataset['test'].num_rows,downsample_factor))

Using custom data configuration default-22e578e895d47822
Reusing dataset csv (/home/jupyter-genevievegraves/.cache/huggingface/datasets/csv/default-22e578e895d47822/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jupyter-genevievegraves/.cache/huggingface/datasets/csv/default-22e578e895d47822/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-ebffb6372e26d996.arrow
Loading cached processed dataset at /home/jupyter-genevievegraves/.cache/huggingface/datasets/csv/default-22e578e895d47822/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-e67abdb78f2c3222.arrow


2056 rows in the train dataset (100x downsampled).
881 rows in the test dataset (100x downsampled).


<div style=background-color:#EEEEFF>

We've loaded the data as a HuggingFace "dataset", which can be read incrementally from disk (instead of loading the whole dataset into memory at once), and which can be fed nicely into PyTorch DataLoaders when it comes time to train a model.  

Notice that we've mixed together the "real" jokes and the "fake" jokes (whose punchlines are generated by GPT-2) in both our "train" and "test" datasets.  The difference is that all the "real" jokes have score > 0 (because we only selected jokes with at least one upvote), whereas we created all the fake jokes with score = 0.
   
Let's take a quick look at how the data are formatted.

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['setup', 'punchline', 'score'],
        num_rows: 2056
    })
    test: Dataset({
        features: ['setup', 'punchline', 'score'],
        num_rows: 881
    })
})


<div style=background-color:#EEEEFF>

The dataset is stored as a DatasetDict, which has two components, "train" and "test", each of which contain the data rows and the "features" from the CSV.
    
We can look at the first examples of the "real" and the "fake" jokes in our test dataset.

In [4]:
print('A joke with a human-generated punchline:')
print(dataset['test'][0])
print()
print('A joke with a GPT2-generated punchline:')
print([x for x in dataset['test'] if x['score']==0][0])

A joke with a human-generated punchline:
{'setup': 'Did you know Google now has a platform for recording your bowel movements?', 'punchline': "It's called Google Sheets.", 'score': 9}

A joke with a GPT2-generated punchline:
{'setup': 'What do you get when you cross a sheep and a kangaroo?', 'punchline': 'You get the breed to sign something for you! (Kangaroos are typically male, and we only cross males once in a while). So', 'score': 0}


<div style=background-color:#EEEEFF>

Now that we've loaded our data, let's use it to train a model!
    
We're not, however, going to start training from scratch.  
    
Transformer models, each with 100s of millions or even 100s of *billions* of free parameters trained on an enormous corpus of documents, are very expensive to train, in terms of both time and compute costs.  Not only do we not want to wait through and pay for all that compute time, but the carbon footprint of training a state-of-the-art model is [LARGE](https://huggingface.co/course/chapter1/4?fw=pt#transformers-are-big-models).  
    
Instead, we will use [transfer learning](https://towardsdatascience.com/cnn-transfer-learning-fine-tuning-9f3e7c5806b2).  To do that, we start from a pre-trained model, which has already been trained through many epochs on huge document datasets.  We will then do some "fine-tuning" training using our Jokes dataset to produce a classifier that is particularly good at distinguishing human-generated vs. GPT2-generated jokes.
    
Let's start with the [BERT model](https://arxiv.org/pdf/1810.04805.pdf) from Google's AI Language lab.  We're going to use it to do "sequence classification", where the model decides if one sequence (in our case, the punchline) is an appropriate follow-on from a previous sequence (in our case, the setup).

In [5]:
import model_tools as mtools

checkpoint, tokenizer, model = mtools.load_model('bert')

Using checkpoint "bert-base-uncased"


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<div style=background-color:#EEEEFF>

We got some warnings that we've loaded a version of BERT that is not already set up for sequence classification, and that it needs some training to be ready to use this way.  That's okay!  Training is exactly what we are about to do.
    
But first, we need to tokenize the data using BERT's tokenizer, so that our text is encoded using the same token mapping that BERT has been pre-trained to expect.  
    
Two additional processes will happen as part of this tokenization: we'll pad the tokenized strings in each batch to be the same length, so that they can be loaded together into a single PyTorch tensor, and we'll generate the associated attention mask that tells the model which tokens are padding tokens that can be ignored.

In [6]:
# Tokenize the data

import data_tools as dtools

# Use a tokenize function to deal with tokenization and (batch) padding:
#    -- all tokenized strings in a batch need to be padded to the same length 
#       to be loaded into a PyTorch tensor together
def tokenize_function(example):
    full_qa = dtools.joke_as_qa(example['setup'], example['punchline'])
    q = [x[:x.find('Answer:')].strip() for x in full_qa]
    a = [x[x.find('Answer:'):].strip() for x in full_qa]
    return tokenizer(q, a, padding="max_length", max_length=60, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

print(tokenized_datasets)

Loading cached processed dataset at /home/jupyter-genevievegraves/.cache/huggingface/datasets/csv/default-22e578e895d47822/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-f425e4cb13baf4a4.arrow
Loading cached processed dataset at /home/jupyter-genevievegraves/.cache/huggingface/datasets/csv/default-22e578e895d47822/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-4e41e1f440a3c4e0.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'punchline', 'score', 'setup', 'token_type_ids'],
        num_rows: 2056
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'punchline', 'score', 'setup', 'token_type_ids'],
        num_rows: 881
    })
})


<div style=background-color:#EEEEFF>

Notice that the tokenization process has generated and added the token "input_ids", "token_type_ids", and "attention_mask" to our data structure.  These features are what gets input to the model training process.
    
We also need to add the classification labels that we use to distinguish "real" from "fake" jokes.  In our case, the "real" data all have *score>0*, while "fake" data have *score=0*, so we will map everything with *score>0* to have *label=1* and everything with *score=0* to have *label=0*.
    
Finally, we want to drop the input columns from the dataset, now that we've generated the token lists, attention masks, and labels that we will pass to the models.

In [7]:
tokenized_datasets = tokenized_datasets.map(lambda batch: {"labels": [int(x > 0) for x in batch["score"]]}, batched=True)

# Clean up / reformat data to fit into a PyTorch DataLoader
# We don't need the text strings themselves anymore
tokenized_datasets = tokenized_datasets.remove_columns(["setup", "punchline", "score"])
tokenized_datasets.set_format("torch")

print(tokenized_datasets)

Loading cached processed dataset at /home/jupyter-genevievegraves/.cache/huggingface/datasets/csv/default-22e578e895d47822/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-822dfa901264c453.arrow
Loading cached processed dataset at /home/jupyter-genevievegraves/.cache/huggingface/datasets/csv/default-22e578e895d47822/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-0c896bf6cf0df668.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 2056
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 881
    })
})


<div style=background-color:#EEEEFF>

Now we're ready to do the fine-tuning training of our classifier!  The training loop itself is implemented in the *train_classifier()* function defined in [model_tools.py](model_tools.py).
    
We'll train through just 3 epochs with our downsampled training set, just to see how well BERT does with minimal training.  The fake jokes looked pretty different from the real ones, so it shouldn't be too hard!

In [8]:
model = mtools.train_classifier(tokenized_datasets, model, epochs=3)

from datetime import datetime
checkpoint_name = checkpoint.split('/')[-1].split('-')[0]
filename = 'models/ClassifyJokes_{}_{:4.2f}subset_{}'.format(checkpoint_name,1.0/downsample_factor,datetime.now().date())+'.pt'

from torch import save
print('Saving model as {}'.format(filename))
save(model,filename)

Running on cuda:0


 33%|███▎      | 257/771 [00:26<00:53,  9.55it/s]

Epoch: 0, Loss: 0.018, Accuracy: 0.967, F1: 0.968


 67%|██████▋   | 514/771 [00:58<00:27,  9.41it/s]

Epoch: 1, Loss: 0.070, Accuracy: 0.952, F1: 0.951


100%|██████████| 771/771 [01:30<00:00,  9.34it/s]

Epoch: 2, Loss: 0.010, Accuracy: 0.972, F1: 0.971


100%|██████████| 771/771 [01:34<00:00,  8.12it/s]


Saving model as models/ClassifyJokes_bert_0.01subset_2021-12-17.pt


<div style=background-color:#EEEEFF>

As expected, the classifier does pretty well with minimal training: better than 95%.  It may not even be able to improve much with the full training set.  It takes ~30 seconds/epoch to run on 1% of the full training set, so it should take a couple of hours to run 3 epochs with the full training set. 

<div style=background-color:#EEEEFF>

The steps here have been encapsulated in the function *classify_punchlines()*, implemented in [punchline_classifier.py](punchline_classifier.py), so you can run the entire process documented above with the following commands:

In [11]:
# from punchline_classifier import classify_punchlines

# train_files = ['data/short_jokes_train.csv','data/short_jokes_train_fake.csv']
# test_files = ['data/short_jokes_test.csv','data/short_jokes_test_fake.csv']

# model = classify_punchlines(train_files, test_files, downsample=20)

<div style=background-color:#EEEEFF>

Using the entire dataset to train the classifier takes a couple of hours and gives only marginal improvement (~98% accuracy).  If you want to run it, we recommend running it from the command line in a detached screen, as in [2.FakePunchlines](2.FakePunchlines.ipynb).

* `$> screen -S train_class`
* `$> python punchline_classifier.py --train data/short_jokes_train.csv,data/short_jokes_train_fake.csv --test data/short_jokes_test.csv,data/short_jokes_test_fake.csv`

Then "Ctl-a d" to detach.

<div style=background-color:#EEEEFF>

When the model is finished running, we can load it with the following:

In [55]:
from torch import load
model = load(filename)
model.device

device(type='cuda', index=0)

In [56]:
# pred = mtools.classify_punchlines(tokenized_datasets['test'],model)

In [ ]:
labels = list(tokenized_datasets['test']['labels'].squeeze().numpy())

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['labels'] = labels
df['pred'] = pred
df['saved_pred'] = saved_pred

In [ ]:
print(df.groupby(['labels','pred']).size().unstack(fill_value=0))

pred      0    1
labels          
0       431    9
1        16  425


In [ ]:
print(df.groupby(['labels','saved_pred']).size().unstack(fill_value=0))

saved_pred    0    1
labels              
0           432    8
1             8  433
